# 🅿️ Parking Slot Annotation Tool (Colab Version)

Tool untuk mendefinisikan region slot parkir pada gambar.

**Cara Penggunaan:**
1. Upload gambar parkir Anda
2. Klik pada gambar untuk menandai titik-titik polygon slot parkir
3. Simpan hasil anotasi ke `parking_regions.json`
4. Download file JSON untuk digunakan di `ai-service`

## 1. Install Dependencies

In [ ]:
!pip install -q opencv-python-headless matplotlib numpy pillow

## 2. Import Libraries

In [ ]:
import cv2
import json
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Polygon
from matplotlib.collections import PatchCollection
from google.colab import files
from PIL import Image
from IPython.display import display, clear_output
import ipywidgets as widgets

print("✅ Libraries loaded successfully!")

## 3. Upload Gambar Parkir

Upload gambar dari ESP32-CAM atau foto area parkir Anda.

In [ ]:
# Upload image
print("📤 Upload gambar parkir Anda...")
uploaded = files.upload()

# Get uploaded filename
image_filename = list(uploaded.keys())[0]
print(f"\n✅ Uploaded: {image_filename}")

# Load image
image = cv2.imread(image_filename)
image_rgb = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
height, width = image.shape[:2]

print(f"📐 Image size: {width} x {height}")

# Display image
plt.figure(figsize=(12, 8))
plt.imshow(image_rgb)
plt.title(f"Uploaded Image: {image_filename}")
plt.axis('on')
plt.show()

## 4. Interactive Annotation Tool

🖱️ **Klik pada gambar** untuk menambah titik polygon.

**Tombol:**
- **Next Slot**: Simpan slot saat ini dan mulai slot baru
- **Reset Current**: Hapus titik slot yang sedang dibuat
- **Undo Last Slot**: Hapus slot terakhir yang tersimpan
- **Save JSON**: Simpan semua region ke file

In [ ]:
# Global state
current_points = []
all_regions = []
current_slot_index = 1

# Create figure
fig, ax = plt.subplots(figsize=(14, 10))
plt.subplots_adjust(bottom=0.2)

# Display image
ax.imshow(image_rgb)
ax.set_title(f"🅿️ Defining Slot P{current_slot_index} | Total saved: {len(all_regions)} slots\n(Klik untuk menambah titik)")

# Status text
status_text = ax.text(0.02, 0.98, "", transform=ax.transAxes, 
                       fontsize=10, verticalalignment='top',
                       bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))

def update_display():
    """Redraw the plot with current state"""
    ax.clear()
    ax.imshow(image_rgb)
    
    # Draw saved regions (blue)
    for region in all_regions:
        points = np.array(region['points'])
        polygon = Polygon(points, fill=True, facecolor='blue', 
                         edgecolor='blue', alpha=0.3, linewidth=2)
        ax.add_patch(polygon)
        # Label
        center = points.mean(axis=0)
        ax.text(center[0], center[1], region['code'], 
               fontsize=12, ha='center', va='center',
               color='white', fontweight='bold',
               bbox=dict(boxstyle='round', facecolor='blue', alpha=0.7))
    
    # Draw current points (green)
    if current_points:
        points = np.array(current_points)
        ax.scatter(points[:, 0], points[:, 1], c='lime', s=100, zorder=5, edgecolors='green')
        
        # Draw lines
        if len(current_points) > 1:
            ax.plot(points[:, 0], points[:, 1], 'g-', linewidth=2)
        
        # Close polygon preview if 3+ points
        if len(current_points) >= 3:
            closed_points = np.vstack([points, points[0]])
            ax.plot(closed_points[:, 0], closed_points[:, 1], 'g--', linewidth=2, alpha=0.5)
            polygon = Polygon(points, fill=True, facecolor='lime', alpha=0.2)
            ax.add_patch(polygon)
    
    ax.set_title(f"🅿️ Defining Slot P{current_slot_index} | Total saved: {len(all_regions)} slots\n(Klik untuk menambah titik, minimal 3 titik per slot)")
    
    # Update status
    status = f"Current slot: {len(current_points)} points"
    ax.text(0.02, 0.98, status, transform=ax.transAxes, 
           fontsize=10, verticalalignment='top',
           bbox=dict(boxstyle='round', facecolor='white', alpha=0.8))
    
    fig.canvas.draw_idle()

def on_click(event):
    """Handle mouse click"""
    global current_points
    
    if event.inaxes != ax:
        return
    if event.button != 1:  # Left click only
        return
    
    x, y = int(event.xdata), int(event.ydata)
    current_points.append([x, y])
    print(f"  Added point ({x}, {y}) - Total: {len(current_points)} points")
    update_display()

# Button callbacks
def next_slot(b):
    global current_points, all_regions, current_slot_index
    
    if len(current_points) >= 3:
        slot_code = f"P{current_slot_index}"
        all_regions.append({
            'code': slot_code,
            'points': current_points.copy()
        })
        print(f"✅ Saved slot {slot_code} with {len(current_points)} points")
        current_slot_index += 1
        current_points = []
        update_display()
    else:
        print("⚠️ Need at least 3 points to define a slot!")

def reset_current(b):
    global current_points
    current_points = []
    print("🔄 Reset current slot points")
    update_display()

def undo_last(b):
    global all_regions, current_slot_index
    if all_regions:
        removed = all_regions.pop()
        current_slot_index -= 1
        print(f"↩️ Removed slot {removed['code']}")
        update_display()
    else:
        print("⚠️ No slots to undo!")

def save_json(b):
    if all_regions:
        with open('parking_regions.json', 'w') as f:
            json.dump(all_regions, f, indent=2)
        print(f"💾 Saved {len(all_regions)} regions to parking_regions.json")
        print("\n📋 Content:")
        print(json.dumps(all_regions, indent=2))
    else:
        print("⚠️ No regions to save!")

# Create buttons
btn_next = widgets.Button(description="Next Slot (N)", button_style='success', icon='plus')
btn_reset = widgets.Button(description="Reset Current (R)", button_style='warning', icon='refresh')
btn_undo = widgets.Button(description="Undo Last (U)", button_style='danger', icon='undo')
btn_save = widgets.Button(description="Save JSON (S)", button_style='info', icon='save')

btn_next.on_click(next_slot)
btn_reset.on_click(reset_current)
btn_undo.on_click(undo_last)
btn_save.on_click(save_json)

# Connect click event
fig.canvas.mpl_connect('button_press_event', on_click)

# Display buttons
display(widgets.HBox([btn_next, btn_reset, btn_undo, btn_save]))

plt.show()

print("\n📝 Instructions:")
print("1. Klik pada gambar untuk menambah titik polygon")
print("2. Klik 'Next Slot' setelah menandai 3+ titik")
print("3. Ulangi untuk slot lainnya")
print("4. Klik 'Save JSON' untuk menyimpan")

## 5. Manual Input (Alternative)

Jika interactive tool tidak berfungsi, Anda bisa memasukkan koordinat secara manual:

In [ ]:
# Manual region definition
# Uncomment and modify coordinates as needed

manual_regions = [
    {
        "code": "P1",
        "points": [[100, 100], [200, 100], [200, 250], [100, 250]]
    },
    {
        "code": "P2", 
        "points": [[220, 100], [320, 100], [320, 250], [220, 250]]
    },
    {
        "code": "P3",
        "points": [[340, 100], [440, 100], [440, 250], [340, 250]]
    }
]

# Visualize manual regions
fig, ax = plt.subplots(figsize=(12, 8))
ax.imshow(image_rgb)

colors = plt.cm.Set1(np.linspace(0, 1, len(manual_regions)))

for i, region in enumerate(manual_regions):
    points = np.array(region['points'])
    polygon = Polygon(points, fill=True, facecolor=colors[i], 
                     edgecolor='white', alpha=0.5, linewidth=2)
    ax.add_patch(polygon)
    center = points.mean(axis=0)
    ax.text(center[0], center[1], region['code'],
           fontsize=14, ha='center', va='center',
           color='white', fontweight='bold')

ax.set_title("Manual Regions Preview")
plt.show()

print("\n📋 Manual regions:")
print(json.dumps(manual_regions, indent=2))

In [ ]:
# Save manual regions
use_manual = False  # Set to True to use manual regions instead

regions_to_save = manual_regions if use_manual else all_regions

if regions_to_save:
    with open('parking_regions.json', 'w') as f:
        json.dump(regions_to_save, f, indent=2)
    print(f"✅ Saved {len(regions_to_save)} regions to parking_regions.json")
else:
    print("⚠️ No regions to save!")

## 6. Download parking_regions.json

Download file JSON untuk digunakan di `ai-service` lokal Anda.

In [ ]:
# Download the JSON file
try:
    files.download('parking_regions.json')
    print("✅ Download started!")
    print("\n📁 Copy file ini ke folder ai-service Anda.")
except Exception as e:
    print(f"❌ Error: {e}")
    print("\nCopy manual dari output di atas.")

## 7. Test Detection dengan YOLO (Optional)

Test apakah deteksi kendaraan berfungsi dengan region yang sudah didefinisikan.

In [ ]:
# Install YOLO
!pip install -q ultralytics

In [ ]:
from ultralytics import YOLO

# Load model
print("Loading YOLO model...")
model = YOLO('yolo11n.pt')
print("✅ Model loaded!")

# Run detection
results = model(image_filename, conf=0.5, verbose=False)

# Get vehicle detections
vehicles = []
for result in results:
    for box in result.boxes:
        cls = int(box.cls[0])
        # Vehicle classes: 2=car, 3=motorcycle, 5=bus, 7=truck
        if cls in [2, 3, 5, 7]:
            xyxy = box.xyxy[0].tolist()
            center = ((xyxy[0]+xyxy[2])/2, (xyxy[1]+xyxy[3])/2)
            vehicles.append({
                'box': xyxy,
                'center': center,
                'confidence': float(box.conf[0])
            })

print(f"\n🚗 Detected {len(vehicles)} vehicles")

# Visualize
fig, ax = plt.subplots(figsize=(14, 10))
ax.imshow(image_rgb)

# Draw regions with occupancy status
def point_in_polygon(point, polygon):
    x, y = point
    n = len(polygon)
    inside = False
    p1x, p1y = polygon[0]
    for i in range(1, n + 1):
        p2x, p2y = polygon[i % n]
        if y > min(p1y, p2y):
            if y <= max(p1y, p2y):
                if x <= max(p1x, p2x):
                    if p1y != p2y:
                        xinters = (y - p1y) * (p2x - p1x) / (p2y - p1y) + p1x
                    if p1x == p2x or x <= xinters:
                        inside = not inside
        p1x, p1y = p2x, p2y
    return inside

regions_to_check = manual_regions if use_manual else all_regions

for region in regions_to_check:
    points = np.array(region['points'])
    
    # Check if occupied
    is_occupied = False
    for v in vehicles:
        if point_in_polygon(v['center'], region['points']):
            is_occupied = True
            break
    
    color = 'red' if is_occupied else 'lime'
    status = 'OCCUPIED' if is_occupied else 'EMPTY'
    
    polygon = Polygon(points, fill=True, facecolor=color, 
                     edgecolor='white', alpha=0.4, linewidth=2)
    ax.add_patch(polygon)
    
    center = points.mean(axis=0)
    ax.text(center[0], center[1], f"{region['code']}\n{status}",
           fontsize=10, ha='center', va='center',
           color='white', fontweight='bold',
           bbox=dict(boxstyle='round', facecolor=color, alpha=0.8))

# Draw vehicle boxes
for v in vehicles:
    x1, y1, x2, y2 = v['box']
    rect = plt.Rectangle((x1, y1), x2-x1, y2-y1, 
                         fill=False, edgecolor='yellow', linewidth=2)
    ax.add_patch(rect)
    ax.plot(v['center'][0], v['center'][1], 'yo', markersize=8)

ax.set_title(f"Detection Result: {len(vehicles)} vehicles detected")
plt.show()

# Summary
print("\n📊 Slot Status Summary:")
for region in regions_to_check:
    is_occupied = any(point_in_polygon(v['center'], region['points']) for v in vehicles)
    status = "🔴 OCCUPIED" if is_occupied else "🟢 EMPTY"
    print(f"  {region['code']}: {status}")

---

## ✅ Selesai!

**Langkah selanjutnya:**
1. Download `parking_regions.json`
2. Copy ke folder `ai-service/`
3. Jalankan `python main.py` di lokal

---